In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
raw_data_path = r"C:\Users\pingk\Downloads\fadhli nitip\fadhli nitip\csv\data_rt4_countrybased_raw_nodrop.csv"
filtered_data_path = r"C:\Users\pingk\Downloads\fadhli nitip\fadhli nitip\csv\data_rt4_countrybased_raw_remove2region.csv"
provincial_codes_path = r"C:\Users\pingk\Downloads\fadhli nitip\provincial code.xlsx" 
                                 
raw_data = pd.read_csv(raw_data_path)
filtered_data = pd.read_csv(filtered_data_path)
province_codes = pd.read_excel(provincial_codes_path)

In [10]:
raw_data.head()

,prov_char,country_code,country_char,wavenumber,4000.122,3999.64,3999.158,3998.676,3998.194,3997.712,...,653.75,653.268,652.786,652.304,651.822,651.34,650.858,650.376,649.893,16158804.55800001
0,BBX,1,ID,ID-BBX-068-2307-031_001,0.000788,0.000813,0.000840,0.000858,0.000862,0.000843,...,0.011951,0.013370,0.014849,0.016154,0.017089,0.017538,0.017469,0.016946,0.016094,NaN
1,BBX,1,ID,ID-BBX-068-2307-031_002,0.001095,0.001078,0.001086,0.001114,0.001151,0.001180,...,0.012086,0.011914,0.011799,0.011801,0.011938,0.012208,0.012600,0.013097,0.013664,NaN
2,BBX,1,ID,ID-BBX-068-2307-031_003,0.000099,0.000153,0.000288,0.000470,0.000658,0.000814,...,0.010487,0.011307,0.012372,0.013532,0.014614,0.015469,0.015986,0.016134,0.015954,NaN
3,BBX,1,ID,ID-BBX-068-2307-031_004,0.001685,0.001802,0.001903,0.001975,0.002017,0.002031,...,0.008177,0.008240,0.008484,0.008982,0.009749,0.010722,0.011752,0.012653,0.013254,NaN
4,BBX,1,ID,ID-BBX-068-2307-031_005,0.002118,0.002196,0.002266,0.002324,0.002369,0.002404,...,0.009831,0.010143,0.010621,0.011171,0.011728,0.012274,0.012824,0.013402,0.014003,NaN


In [11]:
def data_inspection(df, name):
    print(f"--- {name} ---")
    print("Info:")
    print(df.info())
    print("\nMissing Values:")
    print(df.isna().sum())
    print("\nDuplicates:")
    print(df.duplicated().sum())
    print("\nSample Counts per Country:")
    print(df['country_char'].value_counts())
    print("\nSample Counts per Province:")
    print(df['prov_char'].value_counts())

# Inspect raw and filtered data
data_inspection(raw_data, "Raw Data")
data_inspection(filtered_data, "Filtered Data")


--- Raw Data ---
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Columns: 6955 entries, prov_char to 16158804.55800001
dtypes: float64(6951), int64(1), object(3)
memory usage: 23.7+ MB
None

Missing Values:
prov_char              0
country_code           0
country_char           0
wavenumber             0
4000.122               0
                    ... 
651.34                 0
650.858                0
650.376                0
649.893                0
16158804.55800001    447
Length: 6955, dtype: int64

Duplicates:
0

Sample Counts per Country:
country_char
TH    361
ID     73
MY     13
Name: count, dtype: int64

Sample Counts per Province:
prov_char
SNI    75
CPN    42
TRG    39
KBI    37
NRT    24
SNK    24
NWT    24
BBX    18
KBX    18
PKN    18
SUX    18
RNG    16
STN    15
CCO    13
CBI    12
RIX    11
SGR    11
PTN     7
KRI     6
KTX     5
SSX     3
PNA     3
PLG     3
SKM     3
JHR     2
Name: count, dtype: int64
--- Filtered Data ---
Info:
<class

In [13]:
print("Info:")
raw_data.info()

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Columns: 6955 entries, prov_char to 16158804.55800001
dtypes: float64(6951), int64(1), object(3)
memory usage: 23.7+ MB


float64:

    This stands for a 64-bit floating-point number.
    It can represent decimal numbers and is capable of containing a very wide range of values, including very small and very large numbers, with a high degree of precision.
    It is commonly used for representing continuous values, such as measurements and calculations that require precision.
    1,2,3

int64:

    This stands for a 64-bit integer.
    It can store whole numbers (both positive and negative) and has a range large enough to handle most numerical computations that involve whole numbers.
    It's typically used for counting, indexing, or any situation where only whole numbers are needed.
    1.1,1.2,1.3

object:

    In the context of pandas (a popular data manipulation library in Python), object is a flexible data type used to store any kind of Python object.
    It is commonly used to store strings or mixed data types in a column of a DataFrame.
    While flexible, using object type can lead to slower operations compared to the more specialized types like float64 and int64.
    a,b,c

In [15]:
print("\nMissing Values:")
raw_data.isna().sum()


Missing Values:


prov_char              0
country_code           0
country_char           0
wavenumber             0
4000.122               0
                    ... 
651.34                 0
650.858                0
650.376                0
649.893                0
16158804.55800001    447
Length: 6955, dtype: int64

In [16]:
print("\nDuplicates:")
raw_data.duplicated().sum()


Duplicates:


np.int64(0)

In [18]:
print("\nSample Counts per Country:")
raw_data['country_char'].value_counts()


Sample Counts per Country:


country_char
TH    361
ID     73
MY     13
Name: count, dtype: int64

In [20]:
print("\nSample Counts per Province:")
raw_data['prov_char'].value_counts()


Sample Counts per Province:


prov_char
SNI    75
CPN    42
TRG    39
KBI    37
NRT    24
SNK    24
NWT    24
BBX    18
KBX    18
PKN    18
SUX    18
RNG    16
STN    15
CCO    13
CBI    12
RIX    11
SGR    11
PTN     7
KRI     6
KTX     5
SSX     3
PNA     3
PLG     3
SKM     3
JHR     2
Name: count, dtype: int64

In [26]:
ftir_data_on_filtered_data_1 = filtered_data.drop(['country_char', 'prov_char','wavenumber'], axis=1)

In [27]:
import numpy as np

def preprocess_data(df):
    # Standard Normal Variate (SNV)
    snv = (df - df.mean()) / df.std()
    
    # Multiplicative Scatter Correction (MSC)
    mean_spectrum = df.mean(axis=0)
    msc = df.apply(lambda x: (x - x.mean()) / (mean_spectrum - mean_spectrum.mean()), axis=1)
    
    # First Derivative
    first_derivative = df.diff(axis=1).fillna(0)
    
    # Second Derivative
    second_derivative = first_derivative.diff(axis=1).fillna(0)
    
    # Pareto Scaling
    pareto_scaling = (df - df.mean()) / np.sqrt(df.std())
    
    return snv, msc, first_derivative, second_derivative, pareto_scaling

# Apply preprocessing to filtered data
snv, msc, first_derivative, second_derivative, pareto_scaling = preprocess_data(ftir_data_on_filtered_data_1)


In [28]:
from sklearn.feature_selection import VarianceThreshold

def feature_selection(df):
    selector = VarianceThreshold(threshold=0.1)
    return selector.fit_transform(df)

selected_features = feature_selection(ftir_data_on_filtered_data_1)


In [31]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Check the shape of selected features
print("Shape of selected features:", selected_features.shape)

def perform_clustering(data, method='PCA'):
    n_samples, n_features = data.shape
    
    if method == 'PCA':
        n_components = min(2, n_samples, n_features)
        pca = PCA(n_components=n_components)
        reduced_data = pca.fit_transform(data)
        kmeans = KMeans(n_clusters=3)
        clusters = kmeans.fit_predict(reduced_data)
        return reduced_data, clusters
    
    elif method == 'KMeans+PCA':
        kmeans = KMeans(n_clusters=3)
        clusters = kmeans.fit_predict(data)
        n_components = min(2, n_samples, n_features)
        pca = PCA(n_components=n_components)
        reduced_data = pca.fit_transform(data)
        return reduced_data, clusters
    
    elif method == 'Hierarchical+PCA':
        hca = AgglomerativeClustering(n_clusters=3)
        clusters = hca.fit_predict(data)
        n_components = min(2, n_samples, n_features)
        pca = PCA(n_components=n_components)
        reduced_data = pca.fit_transform(data)
        return reduced_data, clusters
    
    elif method == 'UMAP':
        umap = TSNE(n_components=2)
        reduced_data = umap.fit_transform(data)
        kmeans = KMeans(n_clusters=3)
        clusters = kmeans.fit_predict(reduced_data)
        return reduced_data, clusters

# Apply clustering methods and visualize
methods = ['PCA', 'KMeans+PCA', 'Hierarchical+PCA', 'UMAP']
for method in methods:
    reduced_data, clusters = perform_clustering(selected_features, method=method)
    plt.figure(figsize=(10, 7))
    sns.scatterplot(x=reduced_data[:,0], y=reduced_data[:,1], hue=clusters, palette='viridis')
    plt.title(f'Clustering using {method}')
    plt.show()


Shape of selected features: (447, 1)


IndexError: index 1 is out of bounds for axis 1 with size 1

<Figure size 1000x700 with 0 Axes>